## 1. Load data

In [ ]:
from process import *

data = load_data(DatasetDirectory.HEART_DISEASE)

## 2. Prepare datasets

In [4]:
data_preprocessing(data, DatasetDirectory.HEART_DISEASE)
feature_train, label_train, feature_test, label_test = prepare_dataset(data, train_ratio=0.8, test_ratio=0.2)

## 3. Build the decision tree classifier

### 3.1 Build model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion="entropy", random_state=42, max_depth=4)

clf.fit(feature_train, label_train)

label_pred = clf.predict(feature_test)
print("Predicted labels:", label_pred)
print("Actual labels:", label_test)

### 3.2 Visualize the decision tree

In [ ]:
# import graphviz
# from sklearn import tree
# import os
# dot_data = tree.export_graphviz(clf, out_file=None,
#                                  feature_names=[str(i) for i in range(feature_train.shape[1])],
#                                     class_names=[str(i) for i in np.unique(label_train)],
#                                     filled=True, rounded=True,
#                                     special_characters=True)

# graph = graphviz.Source(dot_data)
# directory = os.getcwd()
# graph.render(filename=os.path.join(directory, 'heart_disease_tree'), format='png', cleanup=True)
# print("Decision tree visualization saved as 'heart_disease_tree.png'")

import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

plt.figure(figsize=(20,10))
plot_tree(clf, 
          feature_names=[str(i) for i in range(feature_train.shape[1])],
          class_names=[str(i) for i in np.unique(label_train)],
          filled=True, rounded=True)
plt.show()

## 4. Evaluating the decision tree classifiers

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(label_test, label_pred)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(label_test, label_pred))

print("Confusion Matrix:")
print(confusion_matrix(label_test, label_pred))